# 3. Modeling

Notebook ini mengasumsikan bahwa dataset bersih sudah disimpan sebelumnya sebagai file:

`./Dataset/UsedCarsSA_Clean.csv`

Jika belum ada, jalankan terlebih dahulu notebook notebook 02 untuk menghasilkan file tersebut.

In [164]:
# 3.1 Import library dan load dataset bersih
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Load dataset bersih
data_path_clean = '../Dataset/UsedCarsSA_Clean.csv'
real_df = pd.read_csv(data_path_clean)
df = real_df.copy()

df.head()

,Make,Type,Year,Origin,Color,Options,Engine_Size,Fuel_Type,Gear_Type,Mileage,Region,Price
0,Chrysler,C300,2018,Saudi,Black,Full,5.7,Gas,Automatic,103000,Riyadh,114000.0
1,Nissan,Sunny,2019,Saudi,Silver,Standard,1.5,Gas,Automatic,72418,Riyadh,27500.0
2,Hyundai,Elantra,2019,Saudi,Grey,Standard,1.6,Gas,Automatic,114154,Riyadh,43000.0
3,Hyundai,Elantra,2019,Saudi,Silver,Semi Full,2.0,Gas,Automatic,41912,Riyadh,59500.0
4,Honda,Accord,2018,Saudi,Navy,Full,1.5,Gas,Automatic,39000,Riyadh,72000.0


## 3.2 Menyiapkan Fitur dan Target

In [165]:
# Definisikan target dan fitur
target = 'Price'
X = df.drop(columns=[target])
y = df[target]

In [166]:
y.head()

0    114000.0
1     27500.0
2     43000.0
3     59500.0
4     72000.0
Name: Price, dtype: float64

Pada tahap ini, kolom **Price** ditetapkan sebagai **target (y)** karena merepresentasikan harga mobil yang ingin diprediksi, sedangkan **fitur (X)** terdiri dari seluruh kolom selain Price yang menggambarkan karakteristik kendaraan. Fitur tersebut mencakup atribut **kategorikal** seperti *Make, Type, Origin, Color, Options, Fuel_Type, Gear_Type,* dan *Region*, serta atribut **numerik** seperti *Year, Engine_Size,* dan *Mileage*. Kombinasi fitur-fitur ini diharapkan mampu menangkap faktor utama yang memengaruhi harga mobil bekas, mulai dari spesifikasi teknis, usia kendaraan, hingga preferensi pasar berdasarkan lokasi.

## 3.3 Train–Test Split

In [167]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

X_train.shape, X_test.shape

((3824, 11), (957, 11))

Dataset dibagi menjadi **data latih (training)** dan **data uji (testing)** dengan proporsi **80% : 20%** menggunakan `train_test_split`, di mana `random_state=42` memastikan hasil pembagian konsisten dan dapat direproduksi. Data latih digunakan untuk membangun dan melatih model, sementara data uji disisihkan untuk mengevaluasi performa model pada data yang belum pernah dilihat sebelumnya, sehingga memberikan gambaran kemampuan generalisasi model.


## 3.4 Menentukan Kolom Numerik dan Kategorikal

In [168]:
numeric_features = []
for col in ['Year', 'Engine_Size', 'Mileage', 'Car_Age']:
    if col in X_train.columns:
        numeric_features.append(col)

categorical_features = [col for col in X_train.columns if col not in numeric_features]

numeric_features, categorical_features[:10]  # tampilkan contoh beberapa fitur kategorikal

(['Year', 'Engine_Size', 'Mileage'],
 ['Make',
  'Type',
  'Origin',
  'Color',
  'Options',
  'Fuel_Type',
  'Gear_Type',
  'Region'])

Langkah ini bertujuan untuk **memisahkan fitur numerik dan kategorikal** secara eksplisit sebelum proses preprocessing. Dari hasil di atas, fitur **numerik** yang teridentifikasi adalah **Year, Engine_Size,** dan **Mileage** (sementara **Car_Age** tidak muncul karena kolom tersebut tidak tersedia di `X_train`), sedangkan fitur **kategorikal** mencakup atribut deskriptif kendaraan seperti **Make, Type, Origin, Color, Options, Fuel_Type, Gear_Type,** dan **Region**. Pemisahan ini penting karena masing-masing tipe fitur memerlukan perlakuan preprocessing yang berbeda—numerik akan diimputasi dan diskalakan, sedangkan kategorikal akan diimputasi dan diubah menjadi representasi numerik melalui one-hot encoding—sehingga pipeline modeling menjadi lebih terstruktur dan robust.


## 3.5 Pipeline Preprocessing

- Fitur numerik: imputasi median lalu scaling standar.
- Fitur kategorikal: imputasi modus lalu one-hot encoding.

Kita gunakan `ColumnTransformer` untuk menggabungkan dua jenis preprocessing ini dalam satu pipeline.

In [169]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

Pipeline preprocessing ini memisahkan perlakuan fitur **numerik** dan **kategorikal** secara sistematis: fitur numerik diimputasi menggunakan **median** untuk mengurangi pengaruh outlier lalu diskalakan dengan **RobustScaler** agar lebih stabil terhadap distribusi yang skewed, sedangkan fitur kategorikal diimputasi dengan nilai **modus** dan dikonversi menjadi representasi numerik menggunakan **One-Hot Encoding** dengan `handle_unknown='ignore'` agar aman terhadap kategori baru pada data uji. Seluruh proses digabungkan dalam `ColumnTransformer` sehingga preprocessing berlangsung konsisten, terintegrasi langsung dengan pipeline model, dan meminimalkan risiko data leakage.

## 3.6 Fungsi Evaluasi Model

In [170]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    nonzero = y_true != 0
    return np.mean(np.abs((y_true[nonzero] - y_pred[nonzero]) / y_true[nonzero])) * 100


def evaluate_regression(model, X_train, y_train, X_test, y_test, name='Model'):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae  = mean_absolute_error(y_test, y_test_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

    return {
        'name': name,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'test_mape': test_mape
    }

Evaluasi model regresi pada fungsi ini difokuskan pada **MAE dan MAPE** untuk menjaga penilaian performa tetap sederhana dan relevan dengan tujuan prediksi harga. **MAE** dihitung pada data latih dan data uji untuk memantau potensi **overfitting** melalui perbandingan kesalahan absolut rata-rata, sedangkan **MAPE** dihitung pada data uji untuk menilai **kesalahan relatif** model dalam bentuk persentase pada data yang belum pernah dilihat sebelumnya. Pendekatan ini memastikan evaluasi berfokus pada akurasi prediksi yang dapat diinterpretasikan secara bisnis tanpa menambahkan metrik yang tidak memberikan insight tambahan.

## 3.7 Mencari Model Regresi Terbaik

In [171]:
models = {
    'Linear Regression': LinearRegression(),

    'Random Forest': RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    ),

    'Gradient Boosting': GradientBoostingRegressor(
        random_state=42
    ),

    'XGBoost': XGBRegressor(
        n_estimators=400,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42,
        n_jobs=-1
    )
}

results = []

for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    pipeline.fit(X_train, y_train)

    metrics = evaluate_regression(
        pipeline,
        X_train, y_train,
        X_test, y_test,
        name
    )

    results.append(metrics)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('test_mae')
results_df



,name,train_mae,test_mae,test_mape
3,XGBoost,7468.418734,11358.776026,108.771998
1,Random Forest,4450.459109,12218.697541,116.812628
0,Linear Regression,11219.457151,13151.334845,111.772372
2,Gradient Boosting,12648.872254,14052.283848,116.684057
